In [31]:
import re
import pandas as pd
import numpy as np
import datetime

In [32]:
def parser(file):
    #Initialize list
    lines = []
    
    #Open File
    with open(file, 'r') as file:
        for line in file:
            lines.append(line.rstrip('\n'))

    #Extract Date
    Date = (re.split('[\s]{4,}', lines[1].strip()))
    Date = Date[0]
    Date = Date.replace('RUN DATE:  ', '')

    #Extract indices to cut list
    indices = [i for i, s in enumerate(lines) if '------' in s]
    indicest = [i for i, s in enumerate(lines) if 'NOTE:' in s]

    #Create list with only well licenses
    df1 = lines[(indices[0]+1):(indicest[0]-1)]

    #Create indexes to slice
    df3 = []
    for line in df1:
        df3.extend(re.split('[\s]{2,}', line.strip()))
    
    #Create list without blank items
    df3 = [x for x in df3 if x != '']

    #Create lists for all 7 elements in the table
    df3 = [df3[x:x+7] for x in range(0, len(df3), 7)]

    #create a dataframe out of the list
    if df3 != []:
        data1 = pd.DataFrame(df3)
        data1.set_axis(['LICENCE NUMBER', 'PERMITTEE', 'SUBT CODE', 'PIPELINE FROM LOC', 'O.D. (MAX)', 'APPROVAL DATE', 'OPERATING KM'], axis = 1, inplace=True)
        data1['DATE'] = Date

        subt = pd.read_csv('SUBT.csv')
        dfm = pd.merge(data1, subt, how='left', left_on='SUBT CODE', right_on='SUBT CODE')
        
    return dfm

In [33]:
import os

from pathlib import Path

directory_in_str = "Data/ST96/"

directory = os.fsencode(directory_in_str) 

columns = ['LICENCE NUMBER', 'PERMITTEE', 'SUBT CODE', 'PIPELINE FROM LOC', 'O.D. (MAX)', 'APPROVAL DATE', 'OPERATING KM', 'DATE']

df = pd.DataFrame(columns=columns)

ext = ['.txt','.TXT']

for file in os.listdir(directory): 
    filename = os.fsdecode(file) 
    if filename.endswith(tuple(ext)):
        print(filename) 
        data = parser(os.path.join(directory_in_str, filename))
        df = df.append(data, ignore_index=True, sort=False)
        continue 
    else: 
        continue

25191.txt
26753.txt
29546.txt
22029.txt
5840.txt
12962.txt
28605.txt
19638.txt
26841.txt
20936.txt
29833.txt
21621.txt
7572.txt
10290.txt
12472.txt
3502.txt
13761.txt
21673.txt
5948.txt
985.txt
5906.txt
21347.txt
16255.txt
17754.txt
25322.txt
29529.txt
14239.txt
27708.txt
3152.txt
18824.txt
4734.txt
4309.txt
16825.txt
32343.txt
6483.txt
8199.txt
10194.txt
23306.txt
21697.txt
18857.txt
29537.txt
18687.txt
25980.txt
11272.txt
11543.txt
14442.txt
18305.txt
2518.txt
5141.txt
12958.txt
23188.txt
7851.txt
11816.txt
7821.txt
22050.txt
22770.txt
3512.txt
14572.txt
22733.txt
17570.txt
18294.txt
8078.txt
30947.txt
23044.txt
25787.txt
10598.txt
6523.txt
30585.txt
10234.txt
18886.txt
31935.txt
21120.txt
8172.txt
8422.txt
5287.txt
1564.txt
4122.txt
31162.txt
12487.txt
12293.txt
27666.txt
8675.txt
428.txt
14275.txt
16710.txt
26832.txt
15290.txt
16975.txt
24659.txt
22125.txt
3394.txt
5177.txt
24657.txt
23028.txt
15694.txt
531.txt
5066.txt
23804.txt
24344.txt
11347.txt
30952.txt
24434.txt
24924.txt
28

In [34]:
df

,LICENCE NUMBER,PERMITTEE,SUBT CODE,PIPELINE FROM LOC,O.D. (MAX),APPROVAL DATE,OPERATING KM,DATE,CODE
0,17610,BELLATRIX EXPLORATION LTD.,OE,35/047/08/5,88.90,2014-07-24,0.00,24 JUL 2014,Oil Well Effluent
1,18463,PEMBINA PIPELINE CORPORATION,CO,33/047/10/5,168.30,2014-07-24,0.44,24 JUL 2014,Crude Oil
2,18511,CANADIAN NATURAL RESOURCES LIMITED,NG,34/050/02/4,88.90,2014-07-24,1.74,24 JUL 2014,Natural Gas
3,21205,CANADIAN NATURAL RESOURCES LIMITED,SW,34/050/02/4,130.00,2014-07-24,1.75,24 JUL 2014,Salt Water
4,21806,CONOCOPHILLIPS CANADA RESOURCES CORP,NG,07/029/08/4,114.30,2014-07-24,0.00,24 JUL 2014,Natural Gas
...,...,...,...,...,...,...,...,...,...
76409,55860,LIGHTSTREAM RESOURCES LTD.,OE,10/069/09/5,168.30,2014-03-17,5.21,17 MAR 2014,Oil Well Effluent
76410,55866,LIGHTSTREAM RESOURCES LTD.,NG,10/069/09/5,114.30,2014-03-17,12.72,17 MAR 2014,Natural Gas
76411,55890,CRESCENT POINT ENERGY CORP.,OE,24/063/10/5,130.00,2014-03-17,3.30,17 MAR 2014,Oil Well Effluent
76412,56015,XTO ENERGY CANADA ULC,NG,27/059/19/5,219.10,2014-03-17,5.80,17 MAR 2014,Natural Gas


In [35]:
#Export to CSV
df.to_csv(r'ST96-Historical.csv', index = False)